# Install and import required libraries

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.models import SequenceTagger
from collections import Counter
from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

warnings.filterwarnings("ignore")

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [ ]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [ ]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-13 06:03:07,926 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-13 06:03:07,929 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-13 06:03:07,931 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-13 06:03:07,935 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-13 06:03:26,590 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 06:03:27,740 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 06:03:29,477 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-13 06:03:29,483 Computing label dictionary. Progress:


448it [00:00, 11513.35it/s]

2023-03-13 06:03:29,570 Dictionary created for label 'upos' with 17 values: NOUN (seen 2275 times), PROPN (seen 1754 times), PUNCT (seen 1390 times), VERB (seen 1024 times), ADP (seen 959 times), PRON (seen 560 times), ADJ (seen 420 times), DET (seen 335 times), NUM (seen 313 times), ADV (seen 311 times), CCONJ (seen 295 times), AUX (seen 188 times), SCONJ (seen 171 times), PART (seen 90 times), SYM (seen 10 times), X (seen 4 times)
Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X


2023-03-13 06:03:58,515 SequenceTagger predicts: Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X
2023-03-13 06:03:58,583 ----------------------------------------------------------------------------------------------------
2023-03-13 06:03:58,585 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-13 06:03:58,587 --------------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  4.41it/s]

...

2023-03-13 06:07:02,099 Evaluating as a multi-label problem: False
2023-03-13 06:07:02,125 DEV : loss 0.595714271068573 - f1-score (micro avg)  0.7925
2023-03-13 06:07:02,137 BAD EPOCHS (no improvement): 0
2023-03-13 06:07:02,138 saving best model


2023-03-13 06:07:19,143 ----------------------------------------------------------------------------------------------------
2023-03-13 06:07:28,004 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  6.46it/s]

2023-03-13 06:07:29,225 Evaluating as a multi-label problem: False
2023-03-13 06:07:29,248 0.8111	0.8111	0.8111	0.8111
2023-03-13 06:07:29,250 
Results:
- F-score (micro) 0.8111
- F-score (macro) 0.5983
- Accuracy 0.8111

By class:
              precision    recall  f1-score   support

        NOUN     0.6754    0.9585    0.7925       241
       PROPN     0.9518    0.7783    0.8564       203
       PUNCT     0.9939    1.0000    0.9969       162
         ADP     0.8190    0.8687    0.8431        99
        VERB     0.9565    0.8148    0.8800       108
        PRON     0.8448    0.9423    0.8909        52
         DET     0.5882    0.6897    0.6349        29
       CCONJ     0.8710    0.8710    0.8710        31
         NUM     0.9630    0.8125    0.8814        32
         ADJ     0.5833    0.1628    0.2545        43
         ADV     0.1333    0.0667    0.0889        30
       SCONJ     0.0909    0.0769    0.0833        13
         AUX     0.8000    0.2667    0.4000        15
        PAR

{'test_score': 0.8110902255639098,
 'dev_score_history': [0.3530239099859353,
  0.469760900140647,
  0.5794655414908579,
  0.6427566807313643,
  0.6842475386779184,
  0.6926863572433193,
  0.7320675105485233,
  0.7341772151898734,
  0.7609001406469761,
  0.7925457102672293],
 'train_loss_history': [2.756191838010725,
  2.1187629673265915,
  1.6557557113289372,
  1.4498117320170933,
  1.2666352137278443,
  1.1459461004028864,
  1.0491514378744182,
  0.9563525202062085,
  0.8902362219495742,
  0.8238862749019086],
 'dev_loss_history': [2.2110302448272705,
  1.602314829826355,
  1.3254345655441284,
  1.1581615209579468,
  0.9660467505455017,
  0.9052001237869263,
  0.7964770197868347,
  0.7553235292434692,
  0.6851597428321838,
  0.595714271068573]}

#2-gram

In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
accusation_tweets = df_tweets[df_tweets["accusation"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in accusation_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in accusation aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:07:35,187 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in accusation aspect (Flair, Indonesian model):
1. NOUN NOUN (3813 occurrences)
2. PROPN PROPN (874 occurrences)
3. NOUN PROPN (348 occurrences)
4. VERB NOUN (269 occurrences)
5. PROPN NOUN (224 occurrences)
6. PROPN VERB (120 occurrences)
7. NOUN ADJ (115 occurrences)
8. PUNCT NOUN (108 occurrences)
9. PROPN PUNCT (100 occurrences)
10. NOUN VERB (90 occurrences)


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
attack_tweets = df_tweets[df_tweets["attack"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in attack_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in attack aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:07:50,223 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in attack aspect (Flair, Indonesian model):
1. NOUN NOUN (237 occurrences)
2. PROPN PROPN (24 occurrences)
3. VERB NOUN (15 occurrences)
4. NOUN VERB (11 occurrences)
5. PROPN NOUN (10 occurrences)
6. NOUN PROPN (10 occurrences)
7. NOUN ADJ (10 occurrences)
8. NOUN PRON (7 occurrences)
9. ADJ NOUN (7 occurrences)
10. ADP NOUN (6 occurrences)


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
loyalty_tweets = df_tweets[df_tweets["loyalty"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in loyalty_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in loyalty aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:07:57,044 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in loyalty aspect (Flair, Indonesian model):
1. NOUN NOUN (270 occurrences)
2. PROPN PROPN (67 occurrences)
3. NOUN PROPN (25 occurrences)
4. VERB NOUN (23 occurrences)
5. PROPN NOUN (17 occurrences)
6. PROPN VERB (12 occurrences)
7. NOUN PRON (11 occurrences)
8. ADP NOUN (7 occurrences)
9. NOUN VERB (6 occurrences)
10. NOUN DET (5 occurrences)


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
threat_tweets = df_tweets[df_tweets["threat"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in threat_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in threat aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:08:05,735 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, DET, NUM, ADV, CCONJ, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in threat aspect (Flair, Indonesian model):
1. NOUN NOUN (496 occurrences)
2. PROPN PROPN (113 occurrences)
3. NOUN PROPN (60 occurrences)
4. VERB NOUN (43 occurrences)
5. PROPN VERB (33 occurrences)
6. PROPN NOUN (22 occurrences)
7. NOUN ADJ (17 occurrences)
8. NOUN VERB (13 occurrences)
9. ADP NOUN (12 occurrences)
10. NOUN PRON (11 occurrences)
